In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Vertex AI Grounding

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/grounding/intro-grouding.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/grounding/intro-grouding.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/grounding/intro-grouding.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10

## Overview

Grounding is a capability that grounds outputs from large-language models (LLMs) to a particular source such as documents or web pages. It allows an LLM to provide output that is based on source materials, which can minimize hallucinations. In most situations, you'll want to to ground the output to factual knowledge that is verified and correct.

In the context of grounding in Vertex AI, there are 2 sources of grounding:
- Google Search, for public web data
- Data stores in Vertex AI Search, which can include your own data in the form of website data, unstructured data, or structured data.

### Grounding use cases

The benefits of grounding are:
- Providing information grounding to prevent hallucination
- Accessing the most up-to-date information
- Improving model quality 

### Objective

In this tutorial, you learn how to {TODO: Complete the sentence explaining briefly what you will learn from the notebook, such as
training, hyperparameter tuning, or serving}:

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI

The steps performed include:

- Configuring the LLM and prompt for various examples
- Setting up a data store in Vertex AI Search with your own data
- Sending example prompts to generative text and chat models in Vertex AI
- Sending example prompts with various levels of grounding (no grounding, web grounding, data store grounding)

## Installation

Install the following packages required to execute this notebook.

In [ ]:
!pip3 install --upgrade --quiet google-cloud-aiplatform

Restart the kernel after installing packages:

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).
1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
# PROJECT_ID = "your-project-id"  # @param {type:"string"}
PROJECT_ID = "koverholt-devrel-355716"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

In [ ]:
from google.colab import auth
auth.authenticate_user()

### Import libraries

In [ ]:
import vertexai
from google.cloud import aiplatform
from vertexai.language_models import TextGenerationModel, ChatModel, GroundingSource

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

## Creating a data store in Vertex AI Search

Follow these steps to create a data store in Vertex AI Search with sample data:

1. Navigate to Vertex AI Search and Conversation in the Google Cloud Console
1. Select the `Data Stores` tab in the sidebar
1. Choose the option to create a `+ New Data Store`
1. Select `Cloud Storage` as the data source
1. Input the URL for ... and select the option for `Unstructured documents`
1. Select `Continue` ...
1. Obtain the Data Store ID and input it below

In [ ]:
DATA_STORE_ID = "google-cloud_1690936150456"

## Examples

- Text generation model with web grounding
- Text generation model with data store grounding
- Chat generation model with web grounding
- Chat generation model with data store grounding

In [ ]:
text_model = TextGenerationModel.from_pretrained("text-bison@001")
chat_model = ChatModel.from_pretrained("chat-bison@001")

## Text generation model

### Configuration

In [ ]:
PROMPT = "What is Vertex AI?"

### No Grounding TextGenerationModel.predict

In [ ]:
res_without = text_model.predict(PROMPT)
res_without.grounding_metadata, res_without

### WebGrounding on TextGenerationModel.predict

In [ ]:
grounding_souce = GroundingSource.WebSearch()
web_res = text_model.predict(
    PROMPT,
    grounding_source=grounding_souce,
)
web_res.grounding_metadata, web_res

### VertexAISearchGrounding on TextGenerationModel.predict

In [ ]:
grounding_souce = GroundingSource.VertexAISearch(data_store_id=data_store_id, location=REGION)
es_res = text_model.predict(
    PROMPT,
    grounding_source=grounding_souce,
)
es_res.grounding_metadata, es_res

## Text generation model (async)

### Configuration

In [ ]:
PROMPT = "What is Vertex AI?"

### No Grounding TextGenerationModel.predict_async

In [ ]:
res_without_async = await text_model.predict_async(PROMPT)
res_without_async.grounding_metadata, res_without_async

### WebGrounding on TextGenerationModel.predict_async

In [ ]:
grounding_souce = GroundingSource.WebSearch()
web_res_async = await text_model.predict_async(
    PROMPT,
    grounding_source=grounding_souce,
)
web_res_async.grounding_metadata, web_res_async

### VertexAISearchGrounding on TextGenerationModel.predict_async

In [ ]:
grounding_souce = GroundingSource.VertexAISearch(data_store_id=data_store_id, location=REGION)
es_res_async = await text_model.predict_async(
    PROMPT,
    grounding_source=grounding_souce,
)
es_res_async.grounding_metadata, es_res_async

## Chat generation model

### Configuration

In [ ]:
PROMPT = "What is Vertex AI?"

### NoGrounding on ChatModel.send_message

In [ ]:
chat = chat_model.start_chat()
res_without = chat.send_message(PROMPT)
res_without.grounding_metadata, res_without

### WebGrounding on ChatModel.send_message

In [ ]:
chat = chat_model.start_chat()
grounding_source = GroundingSource.WebSearch()
web_res_chat = chat.send_message(
    PROMPT,
    grounding_source=grounding_source,
)
web_res_chat.grounding_metadata, web_res_chat

### VertexAISearch Grounding on ChatModel.send_message

In [ ]:
chat = chat_model.start_chat()
grounding_source = GroundingSource.VertexAISearch(data_store_id=data_store_id, location=REGION)
es_res_chat = chat.send_message(
    PROMPT,
    grounding_source=grounding_source,
)
es_res_chat.grounding_metadata, es_res_chat

## Chat generation model (async)

### Configuration

In [ ]:
PROMPT = "What is Vertex AI?"

### NoGrounding on ChatModel.send_message_async

In [ ]:
chat = chat_model.start_chat()
res_without_async = await chat.send_message_async(PROMPT)
res_without_async.grounding_metadata, res_without_async

### WebGrounding on ChatModel.send_message_async

In [ ]:
chat = chat_model.start_chat()
grounding_source = GroundingSource.WebSearch()
web_res_chat_async = await chat.send_message_async(
    PROMPT,
    grounding_source=grounding_source,
)
web_res_chat_async.grounding_metadata, web_res_chat_async

### VertexAISearchGrounding on ChatModel.send_message_async

In [ ]:
chat = chat_model.start_chat()
grounding_source = GroundingSource.VertexAISearch(data_store_id=data_store_id, location=REGION)
es_res_chat_async = await chat.send_message_async(
    PROMPT,
    grounding_source=grounding_source,
)
es_res_chat_async.grounding_metadata, es_res_chat_async

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, since this notebook only makes requests to pretrained LLMs in Vertex AI, there are no additional resources to delete.